# Library Imports

In [1]:
import os
import numpy as np
import pandas as pd
from scipy import io
# from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_auc_score, confusion_matrix, f1_score,\
                            precision_score, recall_score, accuracy_score,\
                            average_precision_score, precision_recall_curve
from imblearn.over_sampling import RandomOverSampler
from src.helper_functions import load_data, get_model_perfs, init_model_perfs,\
                                 save_model, save_model_perfs, check_is_best,\
                                 read_model, evaluate_model_predictions,\
                                 update_model_perfs, check_and_save,\
                                 adjusted_classes
from src.metrics import recall_m, f1

Using TensorFlow backend.
/Users/sean/anaconda3/envs/metis/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/sean/anaconda3/envs/metis/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/sean/anaconda3/envs/metis/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/sean/anaconda3/envs

In [2]:
import keras
from keras.callbacks import Callback
from keras.layers import Dense, Activation, Dropout
keras.__version__

'2.2.5'

In [3]:
%load_ext autoreload
%autoreload 2

# Load Data

In [4]:
x_tr, y_tr, x_te, y_te = load_data()

# Build Neural Network

Following the desciption in section 2.2.4 of the [DeepTox article](https://www.frontiersin.org/articles/10.3389/fenvs.2015.00080/full), I tried to use intermediate values in [Table 2](https://www.frontiersin.org/articles/10.3389/fenvs.2015.00080/full#T2) to build the neural network:

Following [this question/answer](https://datascience.stackexchange.com/questions/45165/how-to-get-accuracy-f1-precision-and-recall-for-a-keras-model) and [this question/answer](https://stackoverflow.com/questions/54065733/how-to-employ-the-scikit-learn-evaluation-metrics-functions-with-keras-in-python) to implement usage of recall in model training:

In [5]:
drop_out=0.5    # DeepTox range: 0.5, 0.2, 0
L2_reg = 0.0001 # Default = 0.01
layers = 3      # DeepTox range: 1, 2, 3, 4
act = 'sigmoid' # Consider sigmoid and tanh
neurons = 1024  # DeepTox range: 1024, 2048, 4096, 8192, 16384
# Info on decay: https://datascience.stackexchange.com/questions/26112/decay-parameter-in-keras-optimizers
decay = 0       # DeepTox range: 10^-4, 10^-5, 10^-6
learn_rate = 0.1  #Research appropriate range
DNN = keras.Sequential([
    keras.layers.InputLayer(input_shape=x_tr.shape[1:],name='Input_Layer')
])
for i in range(1,layers+1):
    DNN.add(Dense(units=neurons, activation=act,\
                  name='h'+str(i)+'_'+act+'_activation',\
                  kernel_regularizer=keras.regularizers.l2(L2_reg)))
    DNN.add(Dropout(rate=drop_out,name='Dropout'+str(i)))
DNN.add(Dense(units=1, activation='sigmoid'))
keras.optimizers.Adam(lr=learn_rate, beta_1=0.9,\
                      beta_2=0.999, decay=decay, amsgrad=False)
DNN.compile(optimizer='adam', loss='binary_crossentropy',\
            metrics=['accuracy',recall_m,f1])
DNN.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
h1_sigmoid_activation (Dense (None, 1024)              1684480   
_________________________________________________________________
Dropout1 (Dropout)           (None, 1024)              0         
_________________________________________________________________
h2_sigmoid_activation (Dense (None, 1024)              1049600   
_________________________________________________________________
Dropout2 (Dropout)           (None, 1024)              0         
_________________________________________________________________
h3_sigmoid_activation (Dense (None, 1024)              1049600   
_________________________________________________________________

# Train DNN's

In [6]:
for target in y_tr.columns[:1]:
    # Determine rows with available data
    rows_tr = np.isfinite(y_tr[target]).values
    rows_te = np.isfinite(y_te[target]).values
    x,y = x_tr[rows_tr], y_tr[target][rows_tr]
    
    # Address Class Imbalance
    x_train, x_val, y_train, y_val = train_test_split(x, y, stratify=y, \
                                                      test_size=0.2, random_state=42)
    ros = RandomOverSampler(random_state=0)
    x_resampled, y_resampled = ros.fit_sample(x_train,y_train)
    
    # Train the DNN
    DNN.fit(
        x_resampled, y_resampled, batch_size=512, epochs=100,\
        validation_data=(x_val,y_val), verbose=1,
        callbacks=[
            keras.callbacks.EarlyStopping(patience=16,verbose=1,\
                                          restore_best_weights=True),
            keras.callbacks.ReduceLROnPlateau(factor=0.5,patience=3,verbose=1)
        ])
    
    # Get predictions, calculate model performance and save info
    p_te = DNN.predict(x_te[rows_te])[:,0]
    y_testing=y_te[target][rows_te]
    auc_te = roc_auc_score(y_testing, p_te)
    print("%15s: %3.5f" % (target, auc_te))

    y_hat_testing=DNN.predict_classes(x_te[rows_te])
    average_precision=average_precision_score(y_testing,p_te)
    mv=evaluate_model_predictions(target,'DNN',0.5,y_testing,y_hat_testing,\
                                  auc_te,average_precision)
    check_and_save(target,mv,DNN,True)
    
    # Find max F1 varying probability threshold, calculate modified performance, save
    precision, recall, thresholds = precision_recall_curve(y_testing,p_te)
    # f1 = 2*precision*recall/(precision+recall)  # Sometimes precision=recall=0!
    p,r,t = zip(*[i for i in zip(precision,recall,np.append(thresholds,1)) if i[0:2]!=(0,0)])
    p,r,t = np.array(p),np.array(r),np.array(t)
    f1 = 2*p*r/(p+r)
    m_idx = np.argmax(f1)
    m_thresh = thresholds[m_idx]
    y_hat_testing_adj=adjusted_classes(p_te,m_thresh)
    mv=evaluate_model_predictions(target,'DNN_modT',m_thresh,y_testing,y_hat_testing_adj,\
                                  auc_te,average_precision)
    check_and_save(target,mv,DNN,True)
    display(get_model_perfs(target))

Instructions for updating:
Use tf.cast instead.
Train on 11934 samples, validate on 1689 samples
Epoch 1/100
11934/11934 [==============================] - 3s 255us/step - loss: 1.1162 - acc: 0.5306 - recall_m: 0.5429 - f1: 0.4656 - val_loss: 0.8558 - val_acc: 0.5488 - val_recall_m: 0.7622 - val_f1: 0.2793
Epoch 2/100
11934/11934 [==============================] - 2s 197us/step - loss: 0.8619 - acc: 0.6115 - recall_m: 0.6207 - f1: 0.6084 - val_loss: 0.7493 - val_acc: 0.7022 - val_recall_m: 0.6792 - val_f1: 0.3424
Epoch 3/100
11934/11934 [==============================] - 2s 204us/step - loss: 0.7862 - acc: 0.6656 - recall_m: 0.6984 - f1: 0.6746 - val_loss: 0.6270 - val_acc: 0.7904 - val_recall_m: 0.6152 - val_f1: 0.3997
Epoch 4/100
11934/11934 [==============================] - 3s 219us/step - loss: 0.7566 - acc: 0.6817 - recall_m: 0.7068 - f1: 0.6854 - val_loss: 0.8354 - val_acc: 0.5198 - val_recall_m: 0.9534 - val_f1: 0.3150
Epoch 5/100
11934/11934 [==============================] - 

,model,threshold,accuracy,precision,recall,f1,auc_roc,avg_precision,confusion_matrix,model_filename
0,RF,0.500000,0.911475,0.756757,0.383562,0.509091,0.905028,0.602102,"[[528, 9], [45, 28]]",RF0.joblib
1,RF_modT,0.235000,0.842623,0.424837,0.890411,0.575221,0.905028,0.602102,"[[449, 88], [8, 65]]",RF_modT0.joblib
2,RF_modT,0.370000,0.903279,0.597222,0.589041,0.593103,0.904939,0.599707,"[[508, 29], [30, 43]]",RF_modT1.joblib
3,DNN,0.500000,0.806557,0.352941,0.739726,0.477876,0.864200,0.521595,"[[438, 99], [19, 54]]",DNN0.h5
4,DNN_modT,0.714105,0.873770,0.479167,0.630137,0.544379,0.864200,0.521595,"[[487, 50], [27, 46]]",DNN_modT0.h5
5,DNN,0.500000,0.811475,0.361842,0.753425,0.488889,0.864697,0.507091,"[[440, 97], [18, 55]]",DNN1.h5
6,DNN_modT,0.665159,0.883607,0.511905,0.589041,0.547771,0.864697,0.507091,"[[496, 41], [30, 43]]",DNN_modT1.h5


# Testing DNN Model Saving.

In [9]:
DNN1 = read_model(target,'DNN1.h5')

ValueError: You are trying to load a weight file containing 4 layers into a model with 0 layers

In [7]:
from joblib import dump, load
filename = dump(DNN, './models/NR_AhR/DNN1y.joblib')

In [8]:
DNN2 = load('./models/NR_AhR/DNN1y.joblib')

ValueError: You are trying to load a weight file containing 4 layers into a model with 0 layers

In [12]:
import pickle
pickle.dump(DNN,open('./models/NR_AhR/DNN1z.pkl', 'wb'))

In [13]:
DNN3=pickle.load(open('./models/NR_AhR/DNN1z.pkl', 'rb'))

ValueError: You are trying to load a weight file containing 4 layers into a model with 0 layers